# COGS 108 - Final Project

# Overview

# Names

Beverly Li  
Jiahao Wang  
Daniel Li  
Saurabh Kanhegaonkar  
Jenny Li  

# Group Member IDs

A13523828   
A13052734   
A15041127   
A15286869   
A14611173

# Research Question

# Background and Prior Work

References (include link)
1) 
2)

# Hypothesis

# Datasets

(Copy this information for each dataset)
Dataset Name:
Link to the dataset:
Number of observations:
1-2 sentences describing each dataset.
If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets

# Setup

In [1]:
%matplotlib inline

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import patsy
import scipy.stats as stats

# Configure libraries
# The seaborn library makes plots look nicer
sns.set()
sns.set_context('talk')

# Don't display too many rows/cols of DataFrames
pd.options.display.max_rows = 11
pd.options.display.max_columns = 8

# Round decimals when displaying DataFrames
pd.set_option('precision', 3)

# for graphs 1-3
treasury_elem = pd.read_csv("./data/Most-Recent-Cohorts-Treasury-Elements.csv", low_memory=False)
df_treasury = pd.DataFrame(treasury_elem)
treasury_elem = pd.read_csv("data/MERGED2016_17_PP.csv", low_memory=False)
df_merged = pd.DataFrame(treasury_elem)
all_data_elem = pd.read_csv("data/Most-Recent-Cohorts-All-Data-Elements.csv", low_memory=False)
df_all_data = pd.DataFrame(all_data_elem)

df_treasury = df_treasury[['UNITID', 'MD_EARN_WNE_P6', 'MD_EARN_WNE_P8', 'MD_EARN_WNE_P10']]
df_all_data = df_all_data[['UNITID', 'ADM_RATE', 'HIGHDEG']]

# for graphs 4-5
majors_data = pd.read_csv('data/degrees-that-pay-back.csv')

# for graphs 6-7
rankings = pd.read_csv("data/cwurData.csv")
df_rank = pd.DataFrame(rankings)
df_rank = df_rank[['institution', 'country', 'national_rank']]

small_data_elem = pd.read_csv("data/salaries-by-region.csv", low_memory=False)
df_small_data = pd.DataFrame(small_data_elem)
df_small_data = df_small_data[['School Name', 'Starting Median Salary', 'Mid-Career Median Salary']]

FileNotFoundError: File b'data/Most-Recent-Cohorts-All-Data-Elements.csv' does not exist

# Data Cleaning

Disclaimer: some schools are not in both datasets, resulting in missing ranks in the graphs

In [ ]:
# clean data for graphs 1-3
df_treasury = df_treasury.dropna()
df_merged = df_merged.dropna()
df_all_data = df_all_data.dropna()

df_treasury = df_treasury[df_treasury.MD_EARN_WNE_P6 != 'PrivacySuppressed']

# remove colleges with highest degree associate and below
df_all_data = df_all_data[df_all_data.HIGHDEG > 2]         

df_new = pd.merge(df_treasury, df_all_data, on=['UNITID'])
df_new['MD_EARN_WNE_P6'] = pd.to_numeric(df_new['MD_EARN_WNE_P6'], downcast='signed')

# clean data for graphs 4-5
majors_data['Starting Median Salary'] = majors_data['Starting Median Salary'].str[1:]
majors_data['Starting Median Salary'] = majors_data['Starting Median Salary'].str.replace(',', '')
majors_data['Starting Median Salary'] = pd.to_numeric(majors_data['Starting Median Salary'])
majors_data['Mid-Career Median Salary'] = majors_data['Mid-Career Median Salary'].str[1:]
majors_data['Mid-Career Median Salary'] = majors_data['Mid-Career Median Salary'].str.replace(',', '')
majors_data['Mid-Career Median Salary'] = pd.to_numeric(majors_data['Mid-Career Median Salary'])
majors_data = majors_data.sort_values(by=['Starting Median Salary'])

# clean data for graphs 6-7
# standardize institution names for more matches between datasets
def fix_name(name):
    name = name.lower().strip()
    if '(' in name:
        name = name[:name.find('(')] + name[name.find(')')+1:]
        name = name.replace(' ', '')
        name = name.replace(',', '')
        name = name.replace('-', '')
        name = name.replace('â€“', '')
        name = name.replace('the', '')
        name = name.replace('&', '')
        name = name.replace('and', '')
        name = name.replace('of', '')
        name = name.replace('at', '')
    return name

# manually match schools that could not be fixed with a general function (40 schools)
def align_name():
    df_rank.loc[220, 'School Name'] = 'University of Michigan'
    df_rank.loc[224, 'School Name'] = 'University of Wisconsin (UW) - Madison'
    df_rank.loc[227, 'School Name'] = 'University of Illinois at Urbana-Champaign (UIUC)'
    df_rank.loc[228, 'School Name'] = 'University of Texas (UT) - Austin'
    df_rank.loc[231, 'School Name'] = 'University of Washington (UW)'
    df_rank.loc[232, 'School Name'] = 'Rutgers University'
    df_rank.loc[245, 'School Name'] = 'Pennsylvania State University (PSU)'
    df_rank.loc[246, 'School Name'] = 'Ohio State University (OSU)'
    df_rank.loc[248, 'School Name'] = 'University of Minnesota'
    df_rank.loc[251, 'School Name'] = 'Purdue University'
    df_rank.loc[304, 'School Name'] = 'Texas A&M University'
    df_rank.loc[312, 'School Name'] = 'Indiana University (IU), Bloomington'
    df_rank.loc[372, 'School Name'] = 'University of South Florida (USF)'
    df_rank.loc[382, 'School Name'] = 'University of Maryland Baltimore County (UMBC)'
    df_rank.loc[395, 'School Name'] = 'University of Missouri - Columbia'
    df_rank.loc[407, 'School Name'] = 'University of Massachusetts (UMass) - Amherst'
    df_rank.loc[415, 'School Name'] = 'Louisiana State University (LSU) '
    df_rank.loc[426, 'School Name'] = 'University of Tennessee'
    df_rank.loc[434, 'School Name'] = 'University of Oklahoma'
    df_rank.loc[438, 'School Name'] = 'University of Texas at San Antonio (UTSA)'
    df_rank.loc[461, 'School Name'] = 'Colorado State University (CSU)'
    df_rank.loc[467, 'School Name'] = 'State University of New York (SUNY) at Buffalo'
    df_rank.loc[495, 'School Name'] = 'Washington State University (WSU)'
    df_rank.loc[497, 'School Name'] = 'University of South Carolina'
    df_rank.loc[534, 'School Name'] = 'University of Nebraska'
    df_rank.loc[564, 'School Name'] = 'Oklahoma State University'
    df_rank.loc[568, 'School Name'] = 'State University of New York (SUNY) at Albany'
    df_rank.loc[640, 'School Name'] = 'University of Mississippi '
    df_rank.loc[668, 'School Name'] = 'University of Hawaii'
    df_rank.loc[676, 'School Name'] = 'University of Arkansas'
    df_rank.loc[736, 'School Name'] = 'University of Wisconsin (UW) - Milwaukee'
    df_rank.loc[758, 'School Name'] = 'University of Missouri - Kansas City (UMKC)'
    df_rank.loc[803, 'School Name'] = 'University of Montana'
    df_rank.loc[901, 'School Name'] = 'University Of Maine'
    df_rank.loc[919, 'School Name'] = 'University of Missouri - St. Louis (UMSL)'
    df_rank.loc[932, 'School Name'] = 'University of Alabama at Huntsville (UAH) '
    df_rank.loc[945, 'School Name'] = 'University of North Dakota '
    df_rank.loc[1012, 'School Name'] = 'University of Massachusetts (UMass) - Lowell '
    df_rank.loc[1046, 'School Name'] = 'University of Massachusetts (UMass) - Boston'

df_rank = df_rank.dropna()
df_rank = df_rank[200:1200]
df_rank = df_rank[df_rank['country'] == 'USA']
df_rank.rename(columns={'institution':'School Name'}, inplace=True)
align_name()
df_rank['School Name'] = df_rank['School Name'].apply(fix_name)

df_small_data = df_small_data.dropna()
df_small_data['School Name'] = df_small_data['School Name'].apply(fix_name)

df_merged = pd.merge(df_small_data, df_rank, on='School Name')
df_merged = df_merged.sort_values(by=['national_rank'])

df_merged['Starting Median Salary'] = df_merged['Starting Median Salary'].str[1:]
df_merged['Starting Median Salary'] = df_merged['Starting Median Salary'].str.replace(',', '')
df_merged['Starting Median Salary'] = pd.to_numeric(df_merged['Starting Median Salary'], downcast='signed')

# Data Analysis & Results

In [ ]:
# first graph: 6 years / admission rate
plt.scatter(df_new['ADM_RATE'], df_new['MD_EARN_WNE_P6'], s=5)
plt.title('Median Earnings 6 years after Entry')
plt.xlabel('Admission Rate for College')
plt.ylabel('Dollars')
plt.yticks(np.arange(10000, 120000, step=12000))

slope, intercept, r_value, p_value, std_err = stats.linregress(df_new['ADM_RATE'], df_new['MD_EARN_WNE_P6'])
print("r value " + str(r_value))
print("p value " + str(p_value))

df_new = df_new[df_new.MD_EARN_WNE_P8 != 'PrivacySuppressed']
df_new['MD_EARN_WNE_P8'] = pd.to_numeric(df_new['MD_EARN_WNE_P8'], downcast='signed')

#second graph: 8 years / admission rate
plt.figure(2)
plt.scatter(df_new['ADM_RATE'], df_new['MD_EARN_WNE_P8'], s=5, color='green')
plt.title('Median Earnings 8 years after Entry')
plt.xlabel('Admission Rate for College')
plt.ylabel('Dollars')
plt.yticks(np.arange(10000, 120000, step=12000))

slope, intercept, r_value, p_value, std_err = stats.linregress(df_new['ADM_RATE'], df_new['MD_EARN_WNE_P8'])
print("r value " + str(r_value))
print("p value " + str(p_value))

df_new = df_new[df_new.MD_EARN_WNE_P10 != 'PrivacySuppressed']
df_new['MD_EARN_WNE_P10'] = pd.to_numeric(df_new['MD_EARN_WNE_P10'], downcast='signed')

#third graph: 10 years / admission rate
plt.figure(3)
plt.scatter(df_new['ADM_RATE'], df_new['MD_EARN_WNE_P10'], s=5, color='red')
plt.title('Median Earnings 10 years after Entry')
plt.xlabel('Admission Rate for College')
plt.ylabel('Dollars')
plt.yticks(np.arange(10000, 120000, step=12000))

slope, intercept, r_value, p_value, std_err = stats.linregress(df_new['ADM_RATE'], df_new['MD_EARN_WNE_P10'])
print("r value " + str(r_value))
print("p value " + str(p_value))

# fourth graph: starting median salary vs major
ms_plot = majors_data.plot(x='Undergraduate Major', y=['Starting Median Salary'], title='Undergraduate Major vs. Starting Median Salary', kind='bar', figsize=(20,7))
ms_plot.set_ylabel('Dollars')

majors_data = majors_data.sort_values(by=['Mid-Career Median Salary'])

# fifth graph: mid-career median salary vs major
ms_plot = majors_data.plot(x='Undergraduate Major', y=['Mid-Career Median Salary'], title='Undergraduate Major vs. Mid-Career Median Salary', kind='bar', figsize=(20,7), color='goldenrod')
ms_plot.set_ylabel('Dollars')

# sixth graph: Ranking vs. Median Income
plt.figure(6)
plt.scatter(df_merged['national_rank'], df_merged['Starting Median Salary'], s=5, color='darkblue')
plt.title('Starting Median Salary and Rank')
plt.xlabel('National Ranking')
plt.ylabel('Dollars')
plt.yticks(np.arange(20000, 90000, step=10000))

# get r_value
slope, intercept, r_value, p_value, std_err = stats.linregress(df_merged['national_rank'], df_merged['Starting Median Salary'])
print("Starting Median Salary and Rank r value: " + str(r_value))
print("Starting Median Salary and Rank p value: " + str(p_value))

axes = plt.gca()
x_vals = np.array(axes.get_xlim())
y_vals = intercept + slope * x_vals
plt.plot(x_vals, y_vals, '-')

df_merged['Mid-Career Median Salary'] = df_merged['Mid-Career Median Salary'].str[1:]
df_merged['Mid-Career Median Salary'] = df_merged['Mid-Career Median Salary'].str.replace(',', '')
df_merged['Mid-Career Median Salary'] = pd.to_numeric(df_merged['Mid-Career Median Salary'], downcast='signed')

# seventh graph: 
plt.figure(7)
plt.scatter(df_merged['national_rank'], df_merged['Mid-Career Median Salary'], s=5, color='darkred')
plt.title('Mid-Career Median Salary and Rank')
plt.xlabel('National Ranking')
plt.ylabel('Dollars')
plt.yticks(np.arange(20000, 200000, step=20000))

# get r_value
slope, intercept, r_value, p_value, std_err = stats.linregress(df_merged['national_rank'], df_merged['Mid-Career Median Salary'])
print("Mid-Career Median Salary and Rank r value: " + str(r_value))
print("Mid-Career Median Salary and Rank p value: " + str(p_value))

axes = plt.gca()
x_vals = np.array(axes.get_xlim())
y_vals = intercept + slope * x_vals
plt.plot(x_vals, y_vals, '-')


# Ethics & Privacy

# Conclusion & Discussion